In [1]:
import torch
import os

c:\Users\Giordano\Anaconda3\envs\MACHINELEARNING\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#only in google colab
!git clone https://github.com/Giordano-Cicchetti/MaskTune_NN/

In [2]:
#only on google colab
!cp MaskTune_NN/MNIST/* /content -r
!rm MaskTune_NN/ -r

"cp" non � riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.
"rm" non � riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
from train import *
train= TrainBaseERM(device)


data\MNISTRGB_BIASED\train
data\MNISTRGB_BIASED\test\original


start creating and saving test dataset of BiasedMnist


data\MNISTRGB_BIASED\test\biased


start creating and saving test dataset of BiasedMnist




In [5]:
train.train_erm(epochs=10)

validation epoch 0: 100%|██████████| 94/94 [00:04<00:00, 20.64it/s, loss=0.704, accuracy=0.501]


lr: 0.01


validation epoch 1: 100%|██████████| 94/94 [00:04<00:00, 20.17it/s, loss=0.698, accuracy=0.514]


lr: 0.01


validation epoch 2: 100%|██████████| 94/94 [00:04<00:00, 20.47it/s, loss=0.699, accuracy=0.506]


...

lr: 0.01


validation epoch 8: 100%|██████████| 94/94 [00:04<00:00, 20.20it/s, loss=0.72, accuracy=0.497] 


lr: 0.01


validation epoch 9: 100%|██████████| 94/94 [00:04<00:00, 20.75it/s, loss=0.722, accuracy=0.497]

lr: 0.01


In [5]:
train.train_erm(resume=True,best_resume_checkpoint_path="best_erm_model.pt",last_resume_checkpoint_path="last_erm_model.pt")

validation epoch 9: 100%|██████████| 94/94 [00:10<00:00,  8.84it/s, loss=0.725, accuracy=0.503]


lr: 0.01


validation epoch 10: 100%|██████████| 94/94 [00:10<00:00,  8.66it/s, loss=0.729, accuracy=0.504]


lr: 0.01


validation epoch 11: 100%|██████████| 94/94 [00:10<00:00,  8.75it/s, loss=0.741, accuracy=0.495]


...

lr: 0.00125


validation epoch 98: 100%|██████████| 94/94 [00:04<00:00, 21.34it/s, loss=2.39, accuracy=0.502]


lr: 0.00125


validation epoch 99: 100%|██████████| 94/94 [00:04<00:00, 21.21it/s, loss=2.39, accuracy=0.502]

lr: 0.00125


In [6]:
train.test(train.test_loader_original,"best_erm_model.pt")

----------testing the model----------


test epoch 1: 100%|██████████| 79/79 [00:04<00:00, 18.06it/s, loss=0.698, accuracy=0.508]

Test accuracy = 0.5076147151898734


In [7]:
train.test(train.test_loader_biased,"best_erm_model.pt")

----------testing the model----------


test epoch 1: 100%|██████████| 79/79 [00:04<00:00, 18.53it/s, loss=0.697, accuracy=0.508]

Test accuracy = 0.5079113924050633


In [ ]:
!rm data -r